In [2]:
import shutup; shutup.please()

In [22]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout, LSTM
from keras.preprocessing.text import Tokenizer
from keras.constraints import maxnorm
import time
import warnings
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical

from tensorflow.keras.layers import Embedding
from collections import Counter
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder,LabelEncoder,MinMaxScaler
)
from sklearn.metrics import(
    classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
)
warnings.filterwarnings("ignore")

In [4]:
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder,LabelEncoder,MinMaxScaler
)

In [129]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Basic functions

In [19]:

# class for Y convertions
class TextLabelEncoderDummy:

    def labelencoder(y_df):
        encoder = LabelEncoder()
        encoder.fit(y_df)
        encoded_Y = encoder.transform(y_df)
        return encoded_Y, encoder


    def encoded_to_dummy(encoded_Y):
        # convert encoder variable to dummy variable
        uniques, ids = np.unique(encoded_Y, return_inverse=True)
        dummy_y = to_categorical(ids, len(uniques))
        # dummy_y = np_utils.to_categorical(encoded_Y)
        return dummy_y, uniques


    def reverse_dummy_to_encoded(y_test):
        reverse_dummy = uniques[y_test.argmax(1)]
        return reverse_dummy


    def reverse_encoded_to_text(reverse_dummy):
        reverse_encoded = encoder.inverse_transform(reverse_dummy)
        return reverse_encoded

## Data Transformation

In [7]:
df = pd.read_csv('../../../datasets/dr-nubel/strain_stress_with_target/full_strain_stress_data.csv')
df.head(2)

,strain,stress,height,tor
0,-0.0135,0.630,2.600000,184
1,0.0272,2.311,2.601381,184


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2365 entries, 0 to 2364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   strain  2365 non-null   float64
 1   stress  2365 non-null   float64
 2   height  2365 non-null   float64
 3   tor     2365 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 74.0 KB


### Removing duplicates

In [9]:
# We will use this method to first take a transpose of our dataset as shown below:
df_T = df.T
df_T.shape

(4, 2365)

In [11]:
# Let's find the total number of duplicate features in our dataset using the sum() method, chained with the duplicated() method as shown below.
print(df_T.duplicated().sum())

0


In [12]:
unique_features = df_T.drop_duplicates(keep='first').T
# bNow, let's print the shape of our new training set without duplicate features:
unique_features.shape

(2365, 4)

In [15]:
# duplicated_features = [dup_col for dup_col in df_T.columns if dup_col not in unique_features.columns]
# duplicated_features

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


### Transform data

In [16]:
# convert dataframe to numpy array
values = df.values
X, y = values[:, 0:3], values[:,3]
# X = X.reshape(len(X), 36, 1)
X = np.asarray(X).astype(np.float64)

In [17]:
X.shape,y.shape

((2365, 3), (2365,))

In [120]:
#reshape input to be [samples, time steps, features]
X_reshape = np.reshape(X, (X.shape[0], 1, X.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [125]:
# Transform
scalarX, scalarY = StandardScaler(), MinMaxScaler()

In [126]:
X_transform = scalarX.fit_transform(X_reshape)

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [55]:
scalarY.fit(y.reshape(-1,1))

MinMaxScaler()

In [58]:
y_transform = scalarY.transform(y.reshape(-1,1))

In [59]:
y_transform

array([[0.27669903],
       [0.27669903],
       [0.27669903],
       ...,
       [0.23786408],
       [0.23786408],
       [0.23786408]])

In [95]:
#reshape input to be [samples, time steps, features]
# X_reshape = np.reshape(X_transform, (X_transform.shape[0], 1, X_transform.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [94]:
trainX.shape,y_transform.shape

((2365, 1, 3), (2365, 1))

### Split data

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X_transform, y_transform,test_size=0.3,random_state=42)

In [97]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,test_size=0.5,random_state=42)

In [99]:
X_train.shape

(1655, 1, 3)

In [105]:
input_dim = X_train.shape[2]
output_dim = 20
maxlen = 6
look_back = 1

In [86]:
input_dim

3

In [108]:
#create and fit the LSTM network
model = Sequential()
model.add(LSTM(17, input_shape=(look_back, 3)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=28, verbose=1)


Epoch 1/10


2023-05-17 08:54:41.825276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-17 08:54:41.827294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-17 08:54:41.828501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

60/60 [==============================] - 2s 3ms/step - loss: 0.2034
Epoch 2/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0811
Epoch 3/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0381
Epoch 4/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0269
Epoch 5/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0246
Epoch 6/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0237
Epoch 7/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0228
Epoch 8/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0221
Epoch 9/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0213
Epoch 10/10
60/60 [==============================] - 0s 3ms/step - loss: 0.0205


In [109]:
import math
from sklearn.metrics import mean_squared_error

In [117]:
xtrain = X_train.reshape(-1,3)
xtest = X_test.reshape(-1,3)

In [115]:
X_test.shape

(355, 1, 3)

## Model Definition

#### Model layer

In [127]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=21, activation='relu'))
    model.add(Dense(40, activation='relu'))
    # model.add(Dense(30, activation='relu'))
    # model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(28, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [101]:
# save model
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 20)          60        
                                                                 
 dense_11 (Dense)            (None, None, 60)          1260      
                                                                 
 lstm_21 (LSTM)              (None, None, 30)          10920     
                                                                 
 dropout_12 (Dropout)        (None, None, 30)          0         
                                                                 
 lstm_22 (LSTM)              (None, None, 10)          1640      
                                                                 
 dropout_13 (Dropout)        (None, None, 10)          0         
                                                                 
 lstm_23 (LSTM)              (None, 5)               

In [ ]:

estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=100, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, S_train, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# print("Model accuracy: %.2f"% (accuracy*100))

In [ ]:
model = baseline_model()
history = model.fit(S_train, y_train, batch_size=100, epochs=10, verbose=0, validation_split=0.33)
# create directory to save model
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb#scrollTo=Yh5Mu0yOgE5J
model.save('saved_model/my_model')
model.save('my_model.h5')

In [74]:
print(history.history.keys())

In [ ]:
# saved_model = load_model('saved_model/my_model')
saved_model = load_model('my_model.h5')
# Check its architecture
saved_model.summary()
# 4. evaluate network
score  = saved_model.evaluate(S_val, y_val, verbose=0)
print('Restored model,Loss: {:5.2f}%,  accuracy: {:5.2f}%'.format(100 * score[0], 100 * score[1]))
# 5. make predictions
predictions = saved_model.predict(S_test, verbose=0)
print(predictions)

In [ ]:
# example of making predictions for a regression problem
from sklearn.datasets import make_regression
# generate regression dataset
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y.reshape(100,1))
X = scalarX.transform(X)
y = scalarY.transform(y.reshape(100,1))
# define and fit the final model
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=1000, verbose=0)
# new instances where we do not know the answer
Xnew, a = make_regression(n_samples=3, n_features=2, noise=0.1, random_state=1)
Xnew = scalarX.transform(Xnew)
# make a prediction
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))a